# RO Configuration Report

**Generated by RO Design MCP Server**

## Input Parameters
- Feed Flow: `{{feed_flow_m3h}}` m³/h
- Target Recovery: `{{target_recovery_pct}}`%
- Membrane Type: `{{membrane_type}}`
- Date: `{{report_date}}`

In [ ]:
# Parameters cell for papermill
feed_flow_m3h = 100
target_recovery_pct = 75
membrane_type = "brackish"
configuration_json = '{}'
report_date = "2024-01-01"

In [ ]:
# Import libraries
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Load configuration
config = json.loads(configuration_json)

## Configuration Summary

In [ ]:
# Display configuration summary
print(f"Array Configuration: {config.get('array_notation', 'N/A')}")
print(f"Total Vessels: {config.get('total_vessels', 0)}")
print(f"Total Membrane Area: {config.get('total_membrane_area_m2', 0):,.0f} m²")
print(f"Achieved Recovery: {config.get('achieved_recovery', 0)*100:.1f}%")
print(f"Number of Stages: {config.get('stage_count', 0)}")

# Check for recycle
recycle_info = config.get('recycle_info', {})
if recycle_info.get('uses_recycle', False):
    print(f"\nConcentrate Recycle: Yes")
    print(f"  Recycle Ratio: {recycle_info.get('recycle_ratio', 0)*100:.1f}%")
    print(f"  Recycle Flow: {recycle_info.get('recycle_flow_m3h', 0):.1f} m³/h")

## Stage-by-Stage Analysis

In [ ]:
# Create stage summary table
stages_data = []
for stage in config.get('stages', []):
    stages_data.append({
        'Stage': stage['stage_number'],
        'Vessels': stage['vessel_count'],
        'Feed (m³/h)': f"{stage['feed_flow_m3h']:.1f}",
        'Permeate (m³/h)': f"{stage['permeate_flow_m3h']:.1f}",
        'Concentrate (m³/h)': f"{stage['concentrate_flow_m3h']:.1f}",
        'Recovery (%)': f"{stage['stage_recovery']*100:.1f}",
        'Flux (LMH)': f"{stage['design_flux_lmh']:.1f}",
        'Flux Ratio (%)': f"{stage['flux_ratio']*100:.0f}"
    })

df_stages = pd.DataFrame(stages_data)
display(df_stages)

## Visualizations

In [ ]:
# Flow distribution plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Flow distribution by stage
stages = [s['stage_number'] for s in config.get('stages', [])]
feeds = [s['feed_flow_m3h'] for s in config.get('stages', [])]
permeates = [s['permeate_flow_m3h'] for s in config.get('stages', [])]
concentrates = [s['concentrate_flow_m3h'] for s in config.get('stages', [])]

x = range(len(stages))
width = 0.25

ax1.bar([i - width for i in x], feeds, width, label='Feed', alpha=0.8)
ax1.bar(x, permeates, width, label='Permeate', alpha=0.8)
ax1.bar([i + width for i in x], concentrates, width, label='Concentrate', alpha=0.8)

ax1.set_xlabel('Stage')
ax1.set_ylabel('Flow Rate (m³/h)')
ax1.set_title('Flow Distribution by Stage')
ax1.set_xticks(x)
ax1.set_xticklabels(stages)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Flux performance
flux_values = [s['design_flux_lmh'] for s in config.get('stages', [])]
flux_ratios = [s['flux_ratio']*100 for s in config.get('stages', [])]

ax2.plot(stages, flux_values, 'o-', markersize=10, linewidth=2, label='Design Flux')
ax2.axhline(y=100, color='r', linestyle='--', alpha=0.5, label='100% Target')

# Add flux ratio labels
for i, (stage, ratio) in enumerate(zip(stages, flux_ratios)):
    ax2.annotate(f'{ratio:.0f}%', 
                xy=(stage, flux_values[i]), 
                xytext=(0, 5), 
                textcoords='offset points',
                ha='center')

ax2.set_xlabel('Stage')
ax2.set_ylabel('Flux (LMH)')
ax2.set_title('Flux Performance by Stage')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Pump Initialization Guide

In [ ]:
# Display pump initialization recommendations
pump_guide = config.get('pump_initialization', {})

if pump_guide:
    print("Approach:", pump_guide.get('approach', 'N/A'))
    print("\nStage-specific recommendations:")
    
    pump_data = []
    for stage in pump_guide.get('stages', []):
        pump_data.append({
            'Stage': stage['stage_number'],
            'Initial Pressure (bar)': f"{stage['initial_pressure_bar']:.1f}",
            'Bounds (bar)': f"{stage['optimization_bounds_bar'][0]:.0f}-{stage['optimization_bounds_bar'][1]:.0f}",
            'Pressure Drop (bar)': f"{stage['pressure_drop_pa']/1e5:.1f}"
        })
    
    df_pumps = pd.DataFrame(pump_data)
    display(df_pumps)
    
    print("\nOptimization notes:")
    for note in pump_guide.get('optimization_notes', []):
        print(f"  • {note}")

## Mass Balance Verification

In [ ]:
# Verify mass balance
total_feed = config.get('summary', {}).get('feed_flow_m3h', feed_flow_m3h)
total_permeate = sum(s['permeate_flow_m3h'] for s in config.get('stages', []))

if recycle_info.get('uses_recycle', False):
    disposal_flow = recycle_info.get('disposal_flow_m3h', 0)
    print(f"System Mass Balance (with recycle):")
    print(f"  IN: Fresh Feed = {total_feed:.1f} m³/h")
    print(f"  OUT: Permeate + Disposal = {total_permeate:.1f} + {disposal_flow:.1f} = {total_permeate + disposal_flow:.1f} m³/h")
    balance_error = abs(total_feed - (total_permeate + disposal_flow))
else:
    final_concentrate = config.get('stages', [{}])[-1].get('concentrate_flow_m3h', 0) if config.get('stages') else 0
    print(f"System Mass Balance:")
    print(f"  IN: Feed = {total_feed:.1f} m³/h")
    print(f"  OUT: Permeate + Concentrate = {total_permeate:.1f} + {final_concentrate:.1f} = {total_permeate + final_concentrate:.1f} m³/h")
    balance_error = abs(total_feed - (total_permeate + final_concentrate))

print(f"  Balance Error: {balance_error:.3f} m³/h {'✓' if balance_error < 0.01 else '✗'}")

## Recommendations

Based on the configuration analysis:

1. **Next Steps:**
   - Run WaterTAP simulation with the generated configuration
   - Optimize pump pressures for minimum LCOW
   - Verify performance with detailed hydraulic analysis

2. **Considerations:**
   - Monitor flux distribution across stages
   - Ensure adequate concentrate flow in final stage
   - Consider anti-scalant dosing for high recovery operations